In [1]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [2]:
#KorQuad 데이터세트 경로(json 파일)

train_data = "/content/drive/MyDrive/Colab Notebooks/KorQuAD_v1.0_train.json"
test_data = "/content/drive/MyDrive/Colab Notebooks/KorQuAD_v1.0_dev.json"

In [3]:
#본문, 질문, 답변 데이터 분리

def read_squad(file):
    with open(file, 'rb') as f :
        squad_dict = json.load(f)
    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                        contexts.append(context)
                        questions.append(question)
                        answers.append(answer)

    return contexts, questions, answers


train_contexts, train_questions, train_answers = read_squad(train_data)
val_contexts, val_questions, val_answers = read_squad(test_data)

In [4]:
print('훈련 데이터의 본문 개수:', len(train_contexts))
print('훈련 데이터의 질문 개수:', len(train_questions))
print('훈련 데이터의 답변 개수:', len(train_answers))
print('테스트 데이터의 본문 개수:', len(val_contexts))
print('테스트 데이터의 질문 개수:', len(val_questions))
print('테스트 데이터의 답변 개수:', len(val_answers))

훈련 데이터의 본문 개수: 60407
훈련 데이터의 질문 개수: 60407
훈련 데이터의 답변 개수: 60407
테스트 데이터의 본문 개수: 5774
테스트 데이터의 질문 개수: 5774
테스트 데이터의 답변 개수: 5774


In [5]:
print("첫번째 샘플 본문")
print(train_contexts[0])

첫번째 샘플 본문
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [6]:
print("첫번째 샘플 질문")
print(train_questions[0])

첫번째 샘플 질문
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [7]:
print("첫번째 샘플 정답")
print(train_answers[0])

첫번째 샘플 정답
{'text': '교향곡', 'answer_start': 54}


In [8]:
#정답에 종료 인덱스 추가하기 (정답 인덱스 + 정답 텍스트 길이)

def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
    #뒤에 공백이 있다면 제거
        answer['text'] = answer['text'].rstrip()

    # 시작 인덱스 + 정답길이 = 종료 인덱스 계산
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
        assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

In [9]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [10]:
print("첫번째 샘플 정답")
print(train_answers[0])
print(train_contexts[0][54:57])

첫번째 샘플 정답
{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}
교향곡


In [11]:
#본문의 질문을 토크나이저의 입력으로 사용(train, val)

tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True,padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [12]:
# 토큰 구성 : [CLS] 본문 토큰화 결과 [SEP] 질문 토큰화 결과 [SEP] [PAD]....[PAD]

#[CLS] : 이후 본문이 토큰화 된 결과
# [SEP] : 첫번째 [SEP]는 토큰을 통해 본문이 종료됨
# [SEP] : 두번째 [SEP]는 토큰을 통해 질문이 종료됨
# [PAD] : 최대 길이까지 반복


print('첫번째 샘플의 토큰화 결과 :', train_encodings[0].tokens)

첫번째 샘플의 토큰화 결과 : ['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현', '##악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '##다', '.

In [13]:
#BERT의 입력이 최대 512길이(본문+질문의 길이가 512보다 길 경우 본문이 잘림, 적을 경우 [PAD]로 구성하여 길이 통일)
print('첫번째 샘플의 길이:', len(train_encodings[0].tokens))


첫번째 샘플의 길이: 512


In [14]:
# [PAD] -> 0으로 변환
print('첫번째 샘플의 정수 인코딩:', train_encodings[0].ids)

첫번째 샘플의 정수 인코딩: [2, 13934, 2236, 2440, 27982, 2259, 21310, 2079, 11994, 3791, 2069, 3790, 1508, 2088, 636, 3800, 2170, 3717, 2052, 9001, 8345, 4642, 2200, 3689, 3657, 2079, 19282, 2069, 1363, 2370, 2259, 936, 2069, 554, 2259, 2062, 18, 1504, 4342, 27982, 2259, 13934, 2196, 2440, 2170, 1195, 23260, 6233, 17370, 2113, 2165, 2069, 809, 1, 3706, 2052, 2181, 8642, 2145, 7334, 2170, 4983, 2371, 4007, 1065, 5917, 2386, 2559, 4443, 2138, 4026, 2259, 11994, 3791, 2079, 15864, 2170, 5487, 2371, 4683, 3605, 18, 3819, 5986, 27135, 1376, 2645, 2203, 2292, 2079, 5872, 2200, 5986, 4152, 2252, 22835, 16706, 2052, 5485, 2205, 2259, 17087, 2079, 19282, 29, 2517, 2069, 881, 2088, 652, 2073, 23404, 2069, 1122, 2886, 13964, 16, 3982, 2052, 9944, 21, 2429, 2170, 11994, 3791, 2079, 1258, 2465, 6233, 24294, 1504, 3967, 2170, 4027, 2052, 5121, 3979, 2069, 18274, 21575, 23548, 575, 2073, 5292, 2085, 7251, 2116, 1415, 2062, 18, 3776, 2079, 942, 2286, 2446, 4196, 2079, 3640, 6509, 636, 2079, 4450, 2170, 10329, 15

In [15]:
# 실제 단어의 위치에는 1, [PAD] 위치에는 0
print('첫번째 샘플의 어텐션 마스크:', train_encodings[0].attention_mask)

첫번째 샘플의 어텐션 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [16]:
#answer_start와 answer_end로부터 bert 토크나이저로 토큰화된 인덱스 탐색 및 함수 저장


def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []


    #토큰화 전 문자의 인덱스(시작, 종료)로부터 토큰화한 후의 문자 인덱스 검출
    for i in tqdm(range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        #시작 인덱스가 비정상인 경우(본문에 정답이 없는 경우)
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        #종료 인덱스가 비정상인 경우(본문에 정답이 없는 경우)
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
                deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [17]:
#정답이 없는 데이터의 경우 제거를 위한 변수생성

deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 388138.65it/s]


In [19]:
print("삭제 예정인 훈련 샘플", deleting_list_for_train)
print("삭제 예정인 테스트 샘플", deleting_list_for_test)

삭제 예정인 훈련 샘플 [711, 726, 728, 729, 761, 765, 767, 768, 805, 2586, 2587, 2722, 2724, 2725, 2731, 3392, 3475, 3478, 3491, 3495, 3498, 3919, 4462, 4465, 4513, 4515, 4565, 4765, 4766, 4772, 4774, 4779, 5334, 6603, 6638, 6639, 6748, 6749, 6750, 6765, 6766, 6771, 6776, 6897, 6898, 6900, 7739, 7741, 9203, 9211, 10880, 11039, 11212, 11727, 11776, 11788, 11789, 11791, 12168, 13708, 13711, 13996, 14460, 14461, 14491, 14724, 14729, 14885, 15764, 15970, 15971, 15973, 15974, 15976, 15977, 15979, 15980, 16080, 17683, 17815, 17828, 18389, 18392, 19045, 19052, 19053, 19195, 19636, 19637, 19638, 19640, 19656, 19761, 19764, 19765, 20614, 20618, 21224, 21243, 21334, 21335, 21338, 21361, 21521, 21522, 22627, 22633, 24003, 24577, 24579, 24580, 24768, 25108, 25176, 25182, 25185, 25186, 25187, 25188, 25448, 25451, 25454, 25457, 25460, 27105, 27112, 27113, 27114, 27159, 27293, 27295, 27555, 27558, 28025, 28438, 28779, 29162, 29189, 29289, 29290, 29855, 31889, 31890, 31891, 31894, 31905, 32050, 32051, 32057, 32

In [20]:
#정답이 없는 본문샘플 제거

def delete_samples(encodings, deleting_list):
    input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
    attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)

    start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
    end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

    X_data = [input_ids, attention_masks]
    y_data = [start_positions, end_positions]

    return X_data, y_data

In [21]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [22]:
print('-------------삭제 전 -------------')
print('훈련 데이터의 샘플의 개수 :', len(train_contexts))
print('테스트 데이터의 샘플의 개수 :', len(val_contexts))
print(' ')
print('-------------삭제 후 -------------')
print('훈련 데이터의 샘플의 개수 :', len(X_train[0]))
print('테스트 데이터의 샘플의 개수 :', len(X_test[0]))

-------------삭제 전 -------------
훈련 데이터의 샘플의 개수 : 60407
테스트 데이터의 샘플의 개수 : 5774
 
-------------삭제 후 -------------
훈련 데이터의 샘플의 개수 : 60140
테스트 데이터의 샘플의 개수 : 5717


In [23]:
from transformers import TFBertModel

class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)

        sequence_output = outputs[0]
        logits = self.qa_outputs(sequence_output)

        start_logits, end_logits = tf.split(logits, 2, axis=-1)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits) #시작 인덱스 분류 활용
        end_probs = self.softmax(end_logits)  #종료 인덱스 분류 활용
        return start_probs, end_probs

In [24]:
#warning 무시
from transformers import logging
logging.set_verbosity_error()

In [25]:
model = TFBertForQuestionAnswering("klue/bert-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=[loss, loss])

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [26]:
history = model.fit(X_train,
                    y_train,
                    epochs=3,
                    verbose=1,
                    batch_size=3)

Epoch 1/3


20047/20047 [==============================] - 1389s 67ms/step - loss: 1.4529 - output_1_loss: 0.6660 - output_2_loss: 0.7868
Epoch 2/3
20047/20047 [==============================] - 1324s 66ms/step - loss: 1.0680 - output_1_loss: 0.4797 - output_2_loss: 0.5882
Epoch 3/3
20047/20047 [==============================] - 1322s 66ms/step - loss: 0.9089 - output_1_loss: 0.4049 - output_2_loss: 0.5040


In [27]:
def predict_test_data_by_idx(idx):

    context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
    question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
    print('본문 :', context)
    print('질문 :', question)
    answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
    print('정답 :',tokenizer.decode(answer_encoded))


    output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
    start = tf.math.argmax(tf.squeeze(output[0]))
    end = tf.math.argmax(tf.squeeze(output[1]))+1

    #예측된 인덱스를 통한 정답 추출
    answer_encoded = X_test[0][idx][start:end]
    print('예측     :',tokenizer.decode(answer_encoded))
    print('----------------------------------------')


In [28]:
for i in range(0, 10):
    predict_test_data_by_idx(i)

본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 : 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? 
정답 : 1989년 2월 15일
예측     : 1989년 2월 15일
----------------------------------------
본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대

In [29]:
model.save_weights('/content/drive/MyDrive/Colab Notebooks/mrc_model_weights.h5')

In [30]:
### 여기부턴 학습된 기계독해 모델, 가중치 파일로 저장한 후, 불러와서 테스트 코드

In [31]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import TFBertModel

In [32]:
class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)

        sequence_output = outputs[0]
        logits = self.qa_outputs(sequence_output)

        start_logits, end_logits = tf.split(logits, 2, axis=-1)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits) #시작 인덱스 분류 활용
        end_probs = self.softmax(end_logits)  #종료 인덱스 분류 활용
        return start_probs, end_probs

# 모델 생성
model = TFBertForQuestionAnswering("klue/bert-base")

# 모델 호출 (임의의 입력을 사용하여 초기화)
dummy_input = (tf.constant([[1, 2, 3]], dtype=tf.int32), tf.constant([[1, 1, 1]], dtype=tf.int32))
model(dummy_input)

# 가중치 로드
model.load_weights("/content/drive/MyDrive/Colab Notebooks/mrc_model_weights.h5")

In [33]:
def model_test(test_contexts, test_question):
    tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')
    test_encodings = tokenizer(test_contexts, test_question, truncation=True, padding=True)
    input_ids = np.array(test_encodings['input_ids'])
    attention_masks = np.array(test_encodings['attention_mask'])

    new_data = [input_ids, attention_masks]

    new_context = tokenizer.decode(new_data[0]).split('[SEP] ')[0]
    new_question = tokenizer.decode(new_data[0]).split('[SEP] ')[1]
    print('본문 :', new_context)
    print('질문 :', new_question)

    output = model([tf.constant(new_data[0])[None, :], tf.constant(new_data[1])[None, :]])
    start = tf.math.argmax(tf.squeeze(output[0]))
    end = tf.math.argmax(tf.squeeze(output[1]))+1

    #예측된 인덱스를 통한 정답 추출
    answer_encoded = new_data[0][start:end]
    print('예측 :',tokenizer.decode(answer_encoded))

In [34]:
user = "동서대는 개교 몇 주년을 맞이 했나요?"

In [35]:
text = "1970년 설립된 학교법인 동서학원 산하 동서공과대학으로 1992년 개교[4]한 이후 1996년 지금의 교명으로 변경됨과 동시에 현재는 7개 대학, 3개 계열, 2개 학부 규모의 종합대학으로 성장하였다. 2022년 개교 30주년을 맞이했다."

In [36]:
model_test(text, user)

본문 : [CLS] 1970년 설립된 학교법인 동서학원 산하 동서공과대학으로 1992년 개교 [ 4 ] 한 이후 1996년 지금의 교명으로 변경됨과 동시에 현재는 7개 대학, 3개 계열, 2개 학부 규모의 종합대학으로 성장하였다. 2022년 개교 30주년을 맞이했다. 
질문 : 동서대는 개교 몇 주년인가요? [SEP]
예측 : 30주년


In [44]:
import nbformat
from nbconvert import HTMLExporter

# 노트북 파일 경로
notebook_path = "/content/drive/MyDrive/Colab Notebooks/mrc_test_code_241201.ipynb"
html_output_path = "/content/drive/MyDrive/Colab Notebooks/mrc_test_code_241201.html"

# 노트북 파일 읽기
with open(notebook_path, 'r', encoding='utf-8') as notebook_file:
    notebook_content = nbformat.read(notebook_file, as_version=4)

# HTML 변환기 설정
html_exporter = HTMLExporter()
html_exporter.exclude_input = False  # 코드 셀 포함 여부 설정
html_exporter.exclude_output_prompt = True  # 출력 프롬프트 제외

# 위젯 데이터를 무시
html_exporter.register_filter('filter_data_type', lambda x: False)

# HTML로 변환
html_data, _ = html_exporter.from_notebook_node(notebook_content)

# HTML 파일로 저장
with open(html_output_path, 'w', encoding='utf-8') as html_file:
    html_file.write(html_data)

print(f"Notebook이 HTML로 저장되었습니다: {html_output_path}")

KeyError: 'state'